In [1]:
import torch

# Local imports
from train import train_model

Using device: cpu


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 64,
    "max_epochs": 512,
    "seed": 42,
    "data_path": "../data",
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "huber_beta": 0.5,
    "weight_decay": 1e-3,
    "lr": 3e-4,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 96,
    "num_layers": 7,
    "num_attn_heads": 6,
    "dropout": 0.3,
    "pooling_dropout": 0.15,
    "pooling_dim": 96,
}

In [ ]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 181635


Epoch 1/512: 100%|██████████████████████████████████████████████████████████████████████| 906/906 [02:31<00:00,  5.96it/s]


Epoch 1/512: 
Train Loss = 0.77925, Train MSE = 1.51074, Train MAE = 1.00483, Train Acc = 0.55585
Val Loss = 0.72399, Val MSE = 1.33105, Val MAE = 0.94837, Val Acc = 0.57683


Epoch 2/512: 100%|██████████████████████████████████████████████████████████████████████| 906/906 [00:39<00:00, 22.88it/s]


Epoch 2/512: 
Train Loss = 0.73093, Train MSE = 1.37247, Train MAE = 0.95481, Train Acc = 0.58346
Val Loss = 0.73290, Val MSE = 1.37675, Val MAE = 0.95648, Val Acc = 0.57794


...

Epoch 80/512: 100%|█████████████████████████████████████████████████████████████████████| 906/906 [00:40<00:00, 22.56it/s]


Epoch 80/512: 
Train Loss = 0.55437, Train MSE = 0.95570, Train MAE = 0.76831, Train Acc = 0.70378
Val Loss = 0.58446, Val MSE = 1.03401, Val MAE = 0.80158, Val Acc = 0.68936


Epoch 81/512: 100%|█████████████████████████████████████████████████████████████████████| 906/906 [00:40<00:00, 22.59it/s]


Epoch 81/512: 
Train Loss = 0.55479, Train MSE = 0.95782, Train MAE = 0.76933, Train Acc = 0.70468
Val Loss = 0.53688, Val MSE = 0.91373, Val MAE = 0.75093, Val Acc = 0.71918


Epoch 82/512:   0%|                                                                               | 0/906 [00:00<?, ?it/s]